In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, preprocessing, models, layers, utils, losses, optimizers

import matplotlib.pyplot as plt

In [2]:
## TODO: find a dataset

In [3]:
class AlexNet:
    
    def __init__(self, input_shape, num_categories, epochs, batch_size, learning_rate, momentum):
    
        '''
        Intializes the required parameters
        '''
        self._input_shape = input_shape
        self._num_categories = num_categories
        self._epochs = epochs
        self._batch_size = batch_size
        self._learning_rate = learning_rate
        self._momentum = momentum
        
        self._model = None
        self._hist = None
        
    def create_model(self):
            
        '''
        AlexNet model defined in the paper "ImageNet Classification with Deep Convolutional Neural Networks"

        8 layers = 5 convolutional + 3 fully connected 

        Details with ImageNet data:
        224×224×3 input image 
        → 96 kernels of size 11×11×3; stride is 4px; 
        → response norm + max pooling 
        → 256 kernels of size 5×5×48 
        → response norm + max pooling 
        → 384 kernels of size 3×3×256 
        → 384 kernels of size 3×3×192 
        → 256 kernels of size 3×3×192 
        → FC layer of 4096 
        → FC layer of 4096 
        → FC layer of 1000

        The last FC layer gets fed into a 10 node softmax function which outputs probability for each of the 10 classes.

        Arguments: 
            input_shape: the image shape as a tuple (width, height)  
        '''
        self._model = models.Sequential()
        
        #32x32x3
        self._model.add(layers.Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', 
                                      kernel_initializer='he_normal', 
                                      activation='relu', 
                                      input_shape=self._input_shape))
        
        self._model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding= 'valid'))
        
        self._model.add(layers.Conv2D(filters=256, kernel_size=(5, 5), strides=1, kernel_initializer='he_normal', activation='relu'))
        self._model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding= 'valid'))

        self._model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), kernel_initializer= 'he_normal', activation='relu'))

        self._model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), kernel_initializer= 'he_normal', activation='relu'))

        self._model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), kernel_initializer= 'he_normal',  activation='relu'))
        self._model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding= 'valid'))

        self._model.add(layers.Flatten())

        self._model.add(layers.Dense(units=4096, activation='relu'))
        self._model.add(layers.Dropout(0.5))

        self._model.add(layers.Dense(units=4096, activation='relu'))
        self._model.add(layers.Dropout(0.5))
        
        self._model.add(layers.Dense(units=1000, activation='relu'))


    def preprocess_data(self, X_train, Y_train, X_test, Y_test):
        
        X_train = X_train / 255.0 
        X_test = X_test / 255.0
        
        Y_train = utils.to_categorical(Y_train, self._num_categories)
        Y_train = utils.to_categorical(Y_train, self._num_categories)
        
        
        self._datagen = preprocessing.image.ImageDataGenerator(width_shift_range=0.1,
                                                               height_shift_range=0.1,
                                                               horizontal_flip=True)

        return (X_train, X_test), (Y_train, Y_test)
        
        
    def train(self, X_train, Y_train): 
        '''
        compiles and runs the model
        
        '''
        self._model.compile(optimizer=optimizers.Adam(self._momentum), 
                            loss=losses.CategoricalCrossentropy(from_logits=True), 
                            metrics=['accuracy'])
        
        self._hist = self._model.fit_generator(self._datagen.flow(X_train, Y_train, batch_size=self._batch_size), 
                                  steps_per_epoch=len(X_train) / self._batch_size, 
                                  epochs=self._epochs)
        
        return self._hist

In [5]:
alexnet = AlexNet(input_shape=(224, 224, 3), num_categories=10, epochs=10, batch_size=50, learning_rate=0.01, momentum=0.9)

In [6]:
(train_images, train_labels), (test_images, test_labels) = alexnet.preprocess_data(train_images, train_labels, test_images, test_labels)

In [7]:
alexnet.create_model()